## Fit ramberg osgood

In [ ]:
import numpy as np
import paramaterial as pam
from paramaterial import DataSet, DataItem
import pandas as pd
import scipy as sp


In [ ]:
class DEModel:
    def __init__(self, UPL, LPL, YP, bounds):
        # self.E = E
        self.UPL = UPL
        self.LPL = LPL
        self.YP = YP
        self.bounds = bounds
        self.fitted_params = None
        self.fitting_error = None
        self.x_data = None
        self.y_data = None

    def model(self, y, E, K, n):
        x = y/E + K*((y/E) ** n)
        return x

    def fit(self, x_data, y_data, **de_kwargs):
        self.x_data = x_data
        self.y_data = y_data
        result = list(self.de(self.rmse, self.bounds, **de_kwargs))
        # self.fitted_params = result[0]
        # self.fitting_error = result[1]
        return result

    def predict(self, y):
        return self.model(y, *self.fitted_params)

    def rmse(self, w):
        """Root mean squared error."""
        x_pred = self.model(self.y_data, *w)
        # return np.sqrt(sum((self.x_data - x_pred) ** 2)/len(self.x_data))
        return np.sqrt(max((self.x_data - x_pred) ** 2)/len(self.x_data))

    @staticmethod
    def de(fobj, bounds, mut=0.8, crossp=0.7, popsize=20, its=5000):
        """Differential evolution algorithm."""
        dimensions = len(bounds)
        pop = np.random.rand(popsize, dimensions)
        min_b, max_b = np.asarray(bounds).T
        diff = np.fabs(min_b - max_b)
        pop_denorm = min_b + pop*diff
        fitness = np.asarray([fobj(ind) for ind in pop_denorm])
        best_idx = np.argmin(fitness)
        best = pop_denorm[best_idx]
        for i in range(its):
            for j in range(popsize):
                idxs = [idx for idx in range(popsize) if idx != j]
                a, b, c = pop[np.random.choice(idxs, 3, replace=False)]
                mutant = np.clip(a + mut*(b - c), 0, 1)
                cross_points = np.random.rand(dimensions) < crossp
                if not np.any(cross_points):
                    cross_points[np.random.randint(0, dimensions)] = True
                trial = np.where(cross_points, mutant, pop[j])
                trial_denorm = min_b + trial*diff
                f = fobj(trial_denorm)
                if f < fitness[j]:
                    fitness[j] = f
                    pop[j] = trial
                    if f < fitness[best_idx]:
                        best_idx = j
                        best = trial_denorm
            # yield best, fitness[best_idx]
            yield min_b + pop*diff, fitness, best_idx

In [ ]:
ds = DataSet('data/')